## Bibliothek importieren

In [1]:
import serial as ser

ModuleNotFoundError: No module named 'serial'

# Roboter UNBEDINGT senkrecht stellen!

## Seriellen Port öffnen

In [ ]:
s = ser.Serial ('COM9', baudrate = 9600, parity = 'N', bytesize = 8, stopbits = 1, timeout = None)

## Aufgabe 5

In [8]:
import numpy as np
from math import pi, cos, sin, radians

In [9]:
def forward_kinematic_braccio(joint_angles):
    ''' param: joints: current position of all the joints of the braccio '''
    ''' return: current position'''

    result = np.eye(4)
    
    ## initial translation
    result = result @ translation([281, 0, 3])
    
    ## Joint 0 (Base b)
    result = result @ rotate_z(90 - joint_angles[0]) @ translation([0, 0, 71]) 
    
    ## Joint 1 (Shoulder s)
    result = result @ rotate_y(90 - joint_angles[1]) @ translation([0, 0, 126]) 
    
    ## Joint 2 (Elbow e)
    result = result @ rotate_y(90 - joint_angles[2]) @ translation([0, 0, 124]) 
    
    ## Joint 3 (Wrist tilt v)
    result = result @ rotate_y(90 - joint_angles[3]) @ translation([0, 0, 60])

    ## Joint 4 (Wrist rotate w)
    result = result @ rotate_z(90 - joint_angles[4]) @ translation([0, 0, 134]) 
    
    return result

def translation(trans):
    return np.array([
        [1,0,0,trans[0]],
        [0,1,0,trans[1]],
        [0,0,1,trans[2]],
        [0,0,0,1]
    ])

def rotate_z(theta):
    rad = radians(theta)
    return np.array([
        [cos(rad), -sin(rad), 0, 0],
        [sin(rad), cos(rad), 0, 0],
        [0,0,1,0],
        [0,0,0,1]
    ])

def rotate_y(theta):
    rad = radians(theta)
    return np.array([
        [cos(rad),0, sin(rad), 0],
        [0,1,0,0],
        [-sin(rad),0, cos(rad), 0],
        [0,0,0,1]
    ])

def forward_kinematic_braccio_fixed(joint_angles):
    b = joint_angles[0]
    s = joint_angles[1]
    e = 90
    v = 90
    w = joint_angles[2]
    g = 73
    joint_angles_with_fixed = [b, s, e, v, w, g]
    return forward_kinematic_braccio(joint_angles_with_fixed)

In [10]:
result = forward_kinematic_braccio([90, 90, 90, 90, 90, 90])
print(result)

[[  1.   0.   0. 281.]
 [  0.   1.   0.   0.]
 [  0.   0.   1. 518.]
 [  0.   0.   0.   1.]]


In [11]:
def backward_kinematic_braccio(x_soll, max_iter=1000, error=2):
    
    theta_0 = np.array([90, 90, 90])
    theta_i = theta_0
    
    for i in range(max_iter):
        transformation_matrix = forward_kinematic_braccio_fixed(theta_i)
        x_ist = transformation_matrix[0:3,3]
        diff = np.linalg.norm(x_soll - x_ist)
        if diff < error:
            return theta_0
        
        theta_i = theta_i - 0 ## f(x)/f'(x)
    
    raise Exception("Sorry, no solution found")
    

    

In [12]:
x_goal = np.array([0, 0, 0])
backward_kinematic_braccio(x_goal)

Exception: Sorry, no solution found

# Jaccobi

### Closed Form

In [13]:
def db_closed_forward_kinematic_braccio(joint_angles):
    ''' param: joints: current position of all the joints of the braccio '''
    ''' return: current position'''

    result = np.eye(4)
    
    ## initial translation
    result = result @ translation([281, 0, 3])
    
    ## Joint 0 (Base b)
    result = result @ rotate_dz(90 - joint_angles[0]) @ translation([0, 0, 71]) 
    
    ## Joint 1 (Shoulder s)
    result = result @ rotate_y(90 - joint_angles[1]) @ translation([0, 0, 126]) 
    
    ## Joint 2 (Elbow e)
    result = result @ rotate_y(90 - joint_angles[2]) @ translation([0, 0, 124]) 
    
    ## Joint 3 (Wrist tilt v)
    result = result @ rotate_y(90 - joint_angles[3]) @ translation([0, 0, 60])

    ## Joint 4 (Wrist rotate w)
    result = result @ rotate_z(90 - joint_angles[4]) @ translation([0, 0, 134]) 
    
    return result

def translation(trans):
    return np.array([
        [1,0,0,trans[0]],
        [0,1,0,trans[1]],
        [0,0,1,trans[2]],
        [0,0,0,1]
    ])

def rotate_dz(theta):
    rad = radians(theta)
    return np.array([
        [-sin(rad), -cos(rad), 0, 0],
        [cos(rad), -sin(rad), 0, 0],
        [0,0,0,0],
        [0,0,0,0]
    ])

def rotate_z(theta):
    rad = radians(theta)
    return np.array([
        [cos(rad), -sin(rad), 0, 0],
        [sin(rad), cos(rad), 0, 0],
        [0,0,1,0],
        [0,0,0,1]
    ])

def rotate_y(theta):
    rad = radians(theta)
    return np.array([
        [cos(rad),0, sin(rad), 0],
        [0,1,0,0],
        [-sin(rad),0, cos(rad), 0],
        [0,0,0,1]
    ])

In [14]:
result = db_closed_forward_kinematic_braccio([90, 90, 90, 90, 90, 90])
print(result)

[[ 0. -1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


### Numericaly

In [19]:
def db_forward_kinematic_braccio(joint_angles):
    h = 1E-8
    new_joint_angles = [joint_angles[0] + h, joint_angles[1], joint_angles[2], joint_angles[3], joint_angles[4], joint_angles[5]]
    print(joint_angles)
    print(new_joint_angles)
    return (forward_kinematic_braccio(new_joint_angles) - forward_kinematic_braccio(joint_angles)) / h

In [20]:
result = db_forward_kinematic_braccio([90, 90, 90, 90, 90, 90])
print(result)

[90, 90, 90, 90, 90, 90]
[90.00000001, 90, 90, 90, 90, 90]
[[ 0.          0.01745328  0.          0.        ]
 [-0.01745328  0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
